In [86]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import gensim
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance
import statistics


In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ozkon\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
stopword_list = nltk.corpus.stopwords.words('english')

In [7]:
df = pd.read_json(r'C:\Users\ozkon\Downloads\CL_Cup IT_Data_Scince_секция_кейс_VK_датасет\ranking_train.jsonl', lines = 'True')


In [8]:
df_test = pd.read_json(r'C:\Users\ozkon\Downloads\CL_Cup IT_Data_Scince_секция_кейс_VK_датасет\ranking_test.jsonl', lines = 'True')

In [9]:
df.head(5)

,text,comments
0,How many summer Y Combinator fundees decided n...,[{'text': 'Going back to school is not identic...
1,CBS acquires last.fm for $280m,[{'text': 'It will be curious to see where thi...
2,How Costco Became the Anti-Wal-Mart,[{'text': 'I really hate it when people falsel...
3,"Fortune Favors Big Turds | Screw The Money, Th...",[{'text': 'His real point is that something ca...
4,StartupWeekend: 70 Founders Create One Company...,[{'text': 'Looks like someone hasn't read The ...


In [10]:
df_test.head(5)

,text,comments
0,"iOS 8.0.1 released, broken on iPhone 6 models,...",[{'text': 'I&#x27;m still waiting for them to ...
1,Ask HN: How do US HNers get their health insur...,[{'text': 'Get it from your employer. It&#x27;...
2,San Diego Researcher Crowdfunding Patent-Free ...,[{'text': 'What I don&#x27;t understand is why...
3,Rethinking the origins of the universe,[{'text': 'I&#x27;m not a physicist. I imagin...
4,SlackTextViewController: A new growing text in...,[{'text': 'As someone that doesn&#x27;t do iOS...


In [11]:
df_test['comments'].iloc[0]

[{'text': 'I&#x27;m still waiting for them to stabilize wifi on the iPad sith iOS 8. Their quality has really started to slip for me since 10.9 on Mac.',
  'score': None},
 {'text': 'For those who upgraded, no need to do a restore. You can just option-click &quot;Update&quot; and downgrade the OS back to 8.0.0.Instructions: https:&#x2F;&#x2F;gist.github.com&#x2F;locriani&#x2F;f0f5f4f71a28945c3750',
  'score': None},
 {'text': 'Upgraded shortly after it was released and suffered the consequences.  Just was able to restore back down to 8.0I had to turn iMessage off and back on again in order for Apple to re-register my number... until then I was unable to send to any existing contacts.',
  'score': None},
 {'text': 'I think they were under a lot of pressure on the HealthKit front.  That was one of their big flagship iOS 8 features, they got all these app developers to integrate it, then iOS 8 shipped and they had a showstopper bug and wouldn&#x27;t release any apps using HealthKit.',
  '

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88107 entries, 0 to 88106
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      88107 non-null  object
 1   comments  88107 non-null  object
dtypes: object(2)
memory usage: 1.3+ MB


In [13]:
df['text'].value_counts()

Facebook is down                                                               7
Ask HN: Idea Sunday                                                            6
Ask HN: Who's Hiring?                                                          5
Ask HN: What are you working on?                                               5
Ask HN: Can I help you be more awesome today? (No strings. Inquire within.)    4
                                                                              ..
MIT junior dropping out to work on education startup he founded 5 years ago    1
SEC considers letting startups use social networks to raise money              1
Modern Microprocessors: A 90 Minute Guide.                                     1
Ask HN: Rate my weekend project.                                               1
Pay your rent with a Credit or Debit card. No landlord signup required         1
Name: text, Length: 87664, dtype: int64

In [14]:
df[df['text']=='Facebook is down']

,text,comments
34321,Facebook is down,[{'text': 'We don't need this sort of posting ...
57513,Facebook is down,[{'text': 'Not for me (India) This says otherw...
64259,Facebook is down,[{'text': 'One of the things I remember from a...
81373,Facebook is down,[{'text': 'Intermittent outages for me. This i...
85502,Facebook is down,[{'text': 'Not just limited to the web interfa...
87116,Facebook is down,[{'text': 'Let me look in the calendar; is it ...
87118,Facebook is down,[{'text': 'From the #facebook @ freenode:&lt;S...


In [10]:
!pip install nltk

In [4]:
from nltk.tokenize import TweetTokenizer, word_tokenize, wordpunct_tokenize
import nltk

In [15]:
tokenizer = TweetTokenizer()



In [16]:
df['tokenized_posts'] = df['text'].apply(tokenizer.tokenize)

In [17]:
df.head(5)

,text,comments,tokenized_posts
0,How many summer Y Combinator fundees decided n...,[{'text': 'Going back to school is not identic...,"[How, many, summer, Y, Combinator, fundees, de..."
1,CBS acquires last.fm for $280m,[{'text': 'It will be curious to see where thi...,"[CBS, acquires, last.fm, for, $, 280m]"
2,How Costco Became the Anti-Wal-Mart,[{'text': 'I really hate it when people falsel...,"[How, Costco, Became, the, Anti-Wal-Mart]"
3,"Fortune Favors Big Turds | Screw The Money, Th...",[{'text': 'His real point is that something ca...,"[Fortune, Favors, Big, Turds, |, Screw, The, M..."
4,StartupWeekend: 70 Founders Create One Company...,[{'text': 'Looks like someone hasn't read The ...,"[StartupWeekend, :, 70, Founders, Create, One,..."


In [15]:
df.dtypes

text               object
comments           object
tokenized_posts    object
dtype: object

In [18]:
all_comments = []
all_scores = []
post_texts = []
for j in range (len(df['comments'])):
    for i in df['comments'][j]:
        post_texts.append(df['text'][j])
        all_comments.append(i['text'])
        all_scores.append(i['score'])

In [19]:
len(df['comments'])

88107

In [20]:
all_comments

['Going back to school is not identical with giving up. Some founders go back to school and keep working on the startup while there.  However, those do so much worse than the people who work on the startup full-time that going back to school seems, in practice, not too far removed from a death sentence for a startup.Off the top of my head, I\'d guess we\'ve had about 8 startups where the founders went back to school.  It doesn\'t only happen with summer batches.  Founders from winter batches do it too.Usually the reason is that the startup isn\'t doing very well. However, that judgement depends a lot on how determined the founders are.  One reason we now shy away from funding people still in school is that they often unconsciously want the startup to fail, because the idea of dropping out frightens them.A lot of startups look bad after 3 months.  Someone who\'s out of school and has to make it work or get a job in a cubicle will say "don\'t worry, we\'ll figure out how to make it succe

In [21]:
all_comments = pd.DataFrame(all_comments).rename(columns = {0:'all_comments'})

In [22]:
all_scores = pd.DataFrame(all_scores).rename(columns = {0:'all_scores'})

In [23]:
all_comments

,all_comments
0,Going back to school is not identical with giv...
1,There will invariably be those who don't see t...
2,For me school is a way to be connected to what...
3,I guess it really depends on how hungry you ar...
4,I know pollground decided to go back to school...
...,...
440530,Most major banks offer a service called &#x27;...
440531,"It costs 3.25%, or $74.25 for the example of $..."
440532,As many other comments have pointed out almost...
440533,My apartment building uses Yapstone&#x27;s Ren...


In [24]:
all_scores

,all_scores
0,0
1,1
2,2
3,3
4,4
...,...
440530,0
440531,1
440532,2
440533,3


In [25]:
post_texts = pd.DataFrame(post_texts).rename(columns = {0:'post_texts'})
post_texts

,post_texts
0,How many summer Y Combinator fundees decided n...
1,How many summer Y Combinator fundees decided n...
2,How many summer Y Combinator fundees decided n...
3,How many summer Y Combinator fundees decided n...
4,How many summer Y Combinator fundees decided n...
...,...
440530,Pay your rent with a Credit or Debit card. No ...
440531,Pay your rent with a Credit or Debit card. No ...
440532,Pay your rent with a Credit or Debit card. No ...
440533,Pay your rent with a Credit or Debit card. No ...


In [26]:
all_scores.isna().sum()

all_scores    0
dtype: int64

In [27]:
all_comments.isna().sum()

all_comments    0
dtype: int64

In [28]:
comments_and_scores_with_posts = pd.concat([post_texts, all_comments, all_scores], axis=1)

In [29]:
comments_and_scores_with_posts

,post_texts,all_comments,all_scores
0,How many summer Y Combinator fundees decided n...,Going back to school is not identical with giv...,0
1,How many summer Y Combinator fundees decided n...,There will invariably be those who don't see t...,1
2,How many summer Y Combinator fundees decided n...,For me school is a way to be connected to what...,2
3,How many summer Y Combinator fundees decided n...,I guess it really depends on how hungry you ar...,3
4,How many summer Y Combinator fundees decided n...,I know pollground decided to go back to school...,4
...,...,...,...
440530,Pay your rent with a Credit or Debit card. No ...,Most major banks offer a service called &#x27;...,0
440531,Pay your rent with a Credit or Debit card. No ...,"It costs 3.25%, or $74.25 for the example of $...",1
440532,Pay your rent with a Credit or Debit card. No ...,As many other comments have pointed out almost...,2
440533,Pay your rent with a Credit or Debit card. No ...,My apartment building uses Yapstone&#x27;s Ren...,3


In [141]:
# compression_opts = dict(method='zip',
#                         archive_name='out.csv')  
# comments_and_scores.to_csv('out.zip', index=False,
#           compression=compression_opts) 


In [140]:
# comments_and_scores_with_posts.isna().sum()

all_comments    0
post_texts      0
all_scores      0
dtype: int64

In [29]:
comments_and_scores_with_posts['all_scores'].value_counts()

0    88107
1    88107
2    88107
3    88107
4    88107
Name: all_scores, dtype: int64

In [30]:
comments_and_scores_with_posts['tokenized_posts'] = comments_and_scores_with_posts['post_texts'].apply(tokenizer.tokenize)

In [31]:
comments_and_scores_with_posts.head()

,post_texts,all_comments,all_scores,tokenized_posts
0,How many summer Y Combinator fundees decided n...,Going back to school is not identical with giv...,0,"[How, many, summer, Y, Combinator, fundees, de..."
1,How many summer Y Combinator fundees decided n...,There will invariably be those who don't see t...,1,"[How, many, summer, Y, Combinator, fundees, de..."
2,How many summer Y Combinator fundees decided n...,For me school is a way to be connected to what...,2,"[How, many, summer, Y, Combinator, fundees, de..."
3,How many summer Y Combinator fundees decided n...,I guess it really depends on how hungry you ar...,3,"[How, many, summer, Y, Combinator, fundees, de..."
4,How many summer Y Combinator fundees decided n...,I know pollground decided to go back to school...,4,"[How, many, summer, Y, Combinator, fundees, de..."


In [32]:
comments_and_scores_with_posts['tokenized_comments'] = comments_and_scores_with_posts['all_comments'].apply(tokenizer.tokenize)

In [120]:
comments_and_scores_with_posts.head()

,post_texts,all_comments,all_scores,tokenized_posts,tokenized_comments
0,How many summer Y Combinator fundees decided n...,Going back to school is not identical with giv...,0,"[How, many, summer, Y, Combinator, fundees, de...","[Going, back, to, school, is, not, identical, ..."
1,How many summer Y Combinator fundees decided n...,There will invariably be those who don't see t...,1,"[How, many, summer, Y, Combinator, fundees, de...","[There, will, invariably, be, those, who, don'..."
2,How many summer Y Combinator fundees decided n...,For me school is a way to be connected to what...,2,"[How, many, summer, Y, Combinator, fundees, de...","[For, me, school, is, a, way, to, be, connecte..."
3,How many summer Y Combinator fundees decided n...,I guess it really depends on how hungry you ar...,3,"[How, many, summer, Y, Combinator, fundees, de...","[I, guess, it, really, depends, on, how, hungr..."
4,How many summer Y Combinator fundees decided n...,I know pollground decided to go back to school...,4,"[How, many, summer, Y, Combinator, fundees, de...","[I, know, pollground, decided, to, go, back, t..."


In [54]:
compression_opts = dict(method='zip',
                        archive_name='NLP_tokenized.csv')  
comments_and_scores_with_posts.to_csv('out.zip', index=False,
          compression=compression_opts) 

In [107]:
mean_similarities = []
all_similarities = []
similarities = []
for i in range(0,(len(comments_and_scores_with_posts['post_texts'])),5):
    s1 = comments_and_scores_with_posts.post_texts[i]
    for j in range(i,i+5):
        s2 = comments_and_scores_with_posts.all_comments[j]
        cosine_distance_countvectorizer_method(s1, s2)
        def cosine_distance_countvectorizer_method(s1, s2):

            allsentences = [s1 , s2]

            vectorizer = CountVectorizer()
            all_sentences_to_vector = vectorizer.fit_transform(allsentences)
            text_to_vector_v1 = all_sentences_to_vector.toarray()[0].tolist()
            text_to_vector_v2 = all_sentences_to_vector.toarray()[1].tolist()
            cosine = distance.cosine(text_to_vector_v1, text_to_vector_v2)
#             print('Схожесть равна ',round((1-cosine)*100,2),'%')
#             return cosine
            similarities.append(1-cosine)
    all_similarities.append(similarities)
    similarities = []
# for i in range(len(all_similarities)):
#     mean_similarities.append(statistics.mean(all_similarities[i]))
            


In [68]:
comments_and_scores_with_posts['all_scores'].value_counts()

0    88107
1    88107
2    88107
3    88107
4    88107
Name: all_scores, dtype: int64

In [108]:
all_similarities


[[0.45276577544562735,
  0.3459695943463841,
  0.3186943654539355,
  0.4190666247677026,
  0.4717281765248632],
 [0.20573779994945585, 0.08333333333333337, 0.0, 0.0, 0.10206207261596578],
 [0.2952406987830738,
  0.22118697760190442,
  0.14888750009563956,
  0.3803194146278325,
  0.06900655593423544],
 [0.18360464790728237,
  0.20114557032045344,
  0.13648777260815348,
  0.0,
  0.18181818181818177],
 [0.0,
  0.0916270832672289,
  0.11624763874381927,
  0.12700012700019048,
  0.06565321642986133],
 [0.2192395586403375,
  0.12991396049232617,
  0.11785113019775795,
  0.18156825980064073,
  0.17609018126512477],
 [0.0, 0.0, 0.0, 0.0, 0.0],
 [0.06482037235521643,
  0.0,
  0.0977939573758162,
  0.09250652524397363,
  0.08335649113101073],
 [0.32863353450309973,
  0.16151457061744967,
  0.10846522890932808,
  0.3131121455425747,
  0.0],
 [0.0, 0.10540925533894596, 0.24019223070763074, 0.0, 0.13693063937629146],
 [0.046625240412015656,
  0.08006407690254358,
  0.16823164622761322,
  0.13867504

In [109]:
mean_similarities

[]

In [112]:
similarities_dataframe = pd.DataFrame(all_similarities).rename(columns = {0:'comment_0',1:'comment_1',2:'comment_2',3:'comment_3',4:'comment_4'})


In [113]:
similarities_dataframe.head()

,comment_0,comment_1,comment_2,comment_3,comment_4
0,0.452766,0.345970,0.318694,0.419067,0.471728
1,0.205738,0.083333,0.000000,0.000000,0.102062
2,0.295241,0.221187,0.148888,0.380319,0.069007
3,0.183605,0.201146,0.136488,0.000000,0.181818
4,0.000000,0.091627,0.116248,0.127000,0.065653


In [119]:
compression_opts = dict(method='zip',
                        archive_name='out.csv')  
similarities_dataframe.to_csv('similarities_dataframe.zip', index=False,
          compression=compression_opts) 

In [ ]:
# all_similarities = []
# similarities = []
# for i in range(0:len((comments_and_scores_with_posts['post_texts'])+1):5):
#     s1 = comments_and_scores_with_posts.post_texts[i]
#     for j in range(i,i+5):
#         s2 = comments_and_scores_with_posts.all_comments[j]
#         cosine_distance_countvectorizer_method(s1, s2)
#         def cosine_distance_countvectorizer_method(s1, s2):

#             allsentences = [s1 , s2]

#             vectorizer = CountVectorizer()
#             all_sentences_to_vector = vectorizer.fit_transform(allsentences)
#             text_to_vector_v1 = all_sentences_to_vector.toarray()[0].tolist()
#             text_to_vector_v2 = all_sentences_to_vector.toarray()[1].tolist()
#             cosine = distance.cosine(text_to_vector_v1, text_to_vector_v2)
# #             print('Схожесть равна ',round((1-cosine)*100,2),'%')
# #             return cosine
#             similarities.append(cosine)
#             all_similarities.append(similarities)